# Iterator dataset

We need a version of the dataset that is an iterator and that provides a lesser amount of lines for each batch.
This will allow us to study more variations of the EMOS model.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import itertools
import os
import pathlib
import pandas as pd

from smc01.postprocessing.dataset import SMCParquetDataset
from smc01.postprocessing.util import load_checkpoint_from_run, PandasBatchIteratorDataset

In [ ]:
with hydra.initialize_config_module('smc01.postprocessing.conf'):
    cfg = hydra.compose('train', ['experiment=emos_gdps_metar'])

In [ ]:
dataset = hydra.utils.instantiate(cfg.experiment.dataset)

In [ ]:
dataset[0]

In [ ]:
DATA_DIR = pathlib.Path(os.getenv("DATA_DIR"))
DATASET_DIR = DATA_DIR / '2021-12-20-gdps-metar/'

In [ ]:
dataset = SMCParquetDataset(DATASET_DIR, years=[2021])

In [ ]:
iterator_dataset = PandasBatchIteratorDataset(dataset, 50000)

In [ ]:
batch = next(iter(iterator_dataset))
print('len of batch', len(batch.index))

# Test implementation

I want to test that we get the same amount of examples if we use the iterator or not.

In [ ]:
from smc01.postprocessing.train import make_datasets, make_dataloader

In [ ]:
sample_dir = DATA_DIR / 'interpolated/2022-02-08-sample'
with hydra.initialize_config_module('smc01.postprocessing.conf'):
    cfg = hydra.compose('train', [
        'experiment=emos_gdps_metar',
        'experiment.limit_dataframe_size=5000',
        'num_workers=8',
        'experiment.batch_size=1',
        f'experiment.dataset.dataset_dir={sample_dir!s}',
    ])

In [ ]:
train_dataset, *_ = make_datasets(cfg)

In [ ]:
loader = make_dataloader(cfg, train_dataset)


In [ ]:
n_examples = 0
for b in loader:
    n_examples += b['station_id'].shape[0]
    
print(n_examples)

In [ ]:
sample_dir = DATA_DIR / 'interpolated/2022-02-08-sample'
with hydra.initialize_config_module('smc01.postprocessing.conf'):
    cfg_no_limit = hydra.compose('train', [
        'experiment=emos_gdps_metar',
        '~experiment.limit_dataframe_size',
        'num_workers=8',
        'experiment.batch_size=1',
        f'experiment.dataset.dataset_dir={sample_dir!s}',
    ])

In [ ]:
train_dataset_no_limit, *_ = make_datasets(cfg_no_limit)

In [ ]:
loader = make_dataloader(cfg_no_limit, train_dataset_no_limit)

In [ ]:
n_examples = 0
for b in loader:
    n_examples += b['station_id'].shape[0]
print(n_examples)